In [1]:
onedrive = r"C:\Users\mikek\OneDrive"
# %matplotlib ipympl
# %matplotlib qt
# %matplotlib widget
%matplotlib inline

from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray
import xbout
import scipy
import re

# sys.path.append(r"C:\Users\Mike\OneDrive\Project\sdtools")
sys.path.append(os.path.join(onedrive,r"Project\python-packages\sdtools"))
sys.path.append(r'C:\Users\mikek\OneDrive\Project\python-packages')


from gridtools.hypnotoad_tools import *
from gridtools.b2_tools import *
from gridtools.utils import *

from hermes3.fluxes import *
from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from code_comparison.code_comparison import *


%load_ext autoreload
%autoreload 2
%matplotlib tk

print("Done")


st40_phys_viewer can not be loaded from outside Tokamak Energy!

Done


In [3]:
sys.path.append(r"C:\Users\mikek\OneDrive\Project\python-packages\soledge")
import types
import os
import h5py
from math								import sqrt
import numpy							as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot				as pyp
from matplotlib.backends.backend_pdf	import PdfPages

from routines.h5_routines				import h5_read
from routines.intersect_contour			import intersect_2contours
from routines.utils_walls				import get_in_out_walls, plot2d_walls, get_dmax_points_walls
from routines.globals					import DEBUG, KB, BALLOONING_NAMES

from mesh.get_rz_core_sep				import get_rz_core_sep
from mesh.get_rho_in_out_core_sep		import get_rho_in_out_core_sep
from mesh.find_zones_intersections		import find_zones_intersections
from mesh.compute_mesh_intersections	import compute_mesh_intersections

from files.load_soledge_mesh_file		import load_soledge_mesh_file
from files.load_fluxes_files			import load_fluxes_files
from files.load_ions_list				import load_ions_list
from files.load_refpar_file				import load_refpar_file
from files.load_input_file				import load_input_file
from files.load_transports_coefficients	import load_transports_coefficients

# Global balances

In [4]:
import h5py
import os
import numpy								as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot					as pyp
from matplotlib.backends.backend_pdf		import PdfPages
from routines.h5_routines					import h5_read
from files.load_ions_list				import load_ions_list
import pandas as pd

In [7]:
path = r"C:\Users\mikek\OneDrive\Project\collab\tech\compare_data\run_directory"
last=0.
y_ranges=[]
use_time=0
use_pts=0
log_scale=0
plot_all=0
one_plot=0
no_samex=0
save="none"



# Reconstruction

## Setup

In [10]:
print("plot_balances")

if((len(path) > 0) and (path[-1] != "/")): path = path + "/"

#	Read simulation time

try:
    if_glob = h5py.File(path+"Results/globals", "r")
    tempus	= h5_read(if_glob, "tempus",keep_array=False)
    if_glob.close()
except:
    tempus	= 0.

try:
    residuals	 = np.loadtxt(path+"residuals", dtype='f8')
except:
    print("\tError: Not found or unable to read "+ path+"residuals")
    exit()

LenBalRes = residuals.shape[0]

ions	= load_ions_list(path)										#read ions list

nPlasmas = len(ions)
Atoms  = [ions[1][:ions[1].find("+")-1]]
lAtoms = [len(Atoms[-1])]
for i in range(1,nPlasmas):
    if(Atoms[-1] != ions[i][:lAtoms[-1]]):
        Atoms.append(ions[i][:ions[i].find("+")-1])
        lAtoms.append(len(Atoms[-1]))
        
nAtoms = len(Atoms)

balances = []
for i in range(nPlasmas):
    balances.append(np.loadtxt(path+"balances_{:d}".format(i), dtype='f8'))
    LenBalRes = min(balances[0].shape[0], LenBalRes)

t_balances     = balances[0][:,-1]						#Time array

if((last > 0.) and (last <= 10.)):
    i_after = np.where(t_balances >= last)[0]
    if(len(i_after) > 0):
        last = LenBalRes - np.min(i_after)
    else:
        last = LenBalRes
elif(last > 10.):
    last = min(LenBalRes, int(last))
else:
    last = LenBalRes

#	0 = Residual Ne
#	1 = Residual ND
#	2 = Residual Ge
#	3 = Residual GD
#	4 = Residual Te
#	5 = Residual TD

R_iValues = [0,1,nPlasmas,nPlasmas+1,2*nPlasmas,2*nPlasmas+1]
if(use_pts == 0):	R_lines = ["-b","--b","-r","-c","--r","-y","--y"]
else:				R_lines = [".b","ob", ".r",".c","or", ".y","oy"]
R_labels  = ["$Res_Ne*10^{-3}$", "$Res_ND*10^{-3}$", "$Res_Ge*10^{-3}$", "$Res_GD*10^{-3}$","$Res_{Te}*10^{-3}$", "$Res_{TD}*10^{-3}$"]
R_facts   = [1e3,1e3,1e3,1e3,1e3,1e3]

#	3 = Energy input
#	4 = Energy output
#	5 = Energy source
#	6 = Energy radiation
#	7 = Energy variation 
#	8 = Energy stored

E_iValues  = [3,4,5,6,7,8]

if(use_pts == 0): E_lines	= ['-b',  '-g',   '-r',  '-c', '-m',  '-y', '-k',"--m", '--k']
else:			  E_lines	= ['.b',  '.g',   '.r',  '.c', '.m',  '.y', 'k.',"om",  'ok']
E_labels = ["$P_{in}$", "$P_{out}$", "$P_{sou}$", "$P_{rad}$", "$P_{var}$", "$E_{stored}$", "$P_{DIFF}$"]
E_facts = [1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6,1e-6,1e-6]

#	0  = Particle input
#	1  = Particle output
#	2  = Particle source
#	9  = Particle variation
#	10 = Particle stored

N_iValues  = [0,1,2,9,10]
if(use_pts == 0):	N_lines  = ["-b","-g","-r","-m","-y"]
else:				N_lines  = [".b",".g",".r",".m",".y"]
N_labels = ["$\Gamma_{in}*10^{21}$", "$\Gamma_{out}*10^{24}$", "$\Gamma_{sou}*10^{24}$", "$N_{Var}*10^{22}$", "$N_{Stored}*10^{21}$"]
N_facts = [1e-21,1e-24,1e-24,1e-22,1e-21]



plot_balances


## Setup figures

In [12]:
i_plot_file = 0

if(save == "pdf"):	pdf = PdfPages("plot_balances_t={:.3f}.".format(tempus)+save)   #pdf in one file only

nRows		= 4
nCols		= 1
PlotPerFig	= nRows*nCols

nPLots		= 2*(nAtoms)
if(plot_all != 0): nPLots += 2*(nPlasmas-2)

nFigs		= int(nPLots/PlotPerFig)
if(nFigs*PlotPerFig < nPLots): nFigs += 1

if(use_time == 1):
    x_balances = t_balances[-last:]
    x_Labels   = "$t\ (s)$"
else:
    x_balances = np.arange(last,dtype='f8')
    x_Labels    = "$n_{Iter}\ (*10^3)$"

y_Labels	= ["$Residuals$",  "$(MW)&(MJ)$", "$(s^{-1})$", "$(MW)&(MJ)$", "$(MW)&(MJ)$"]

for i in range(int(nPLots/2)-1):
    y_Labels.append("$(MW)&(MJ)$")
    y_Labels.append("$(s^{-1})$")
    
    
Fig = []
Ax  = []
if(one_plot != 1):
    Fig.append(pyp.figure())
    for i in range(3):
        if((i == 0) or (no_samex != 0)):	Ax.append(Fig[-1].add_subplot(3,1,i+1))
        else:								Ax.append(Fig[-1].add_subplot(3,1,i+1, sharex = Ax[0]))
    Fig[-1].tight_layout()
    for i in range(nFigs):	
        Fig.append(pyp.figure())
        for k in range(min(PlotPerFig,nPLots-i*PlotPerFig)):
            if(no_samex != 0):	Ax.append(Fig[-1].add_subplot(nRows,nCols,k+1))
            else:				Ax.append(Fig[-1].add_subplot(nRows,nCols,k+1, sharex = Ax[0]))
        Fig[-1].tight_layout()
else:
    for i in range(nPLots+3):
        Fig.append(pyp.figure())
        if((i == 0) or (no_samex != 0)):	Ax.append(Fig[i].add_subplot(111))
        else:								Ax.append(Fig[i].add_subplot(111, sharex = Ax[0]))

nFigs  += 1													#Add residual figure
nPLots += 3													#Add first page plots

for i in range(len(y_Labels)):
    Ax[i].set_xlabel(x_Labels)
    Ax[i].set_ylabel(y_Labels[i])
    if(log_scale == 0):
        Ax[i].set_yscale('linear')
    else:
        Ax[i].set_yscale('log')

    k = i
    if(i > 0):  k = int((i-1)/2) + 1
    if(i == 3): k = 1 
    if((len(y_ranges) > i) and (y_ranges[k] > 0.)):
        Ax[i].set_ylim((-y_ranges[k], y_ranges[k]))
        Ax[i].autoscale(enable=True,  axis='x', tight=True)
        Ax[i].autoscale(enable=False, axis='y', tight=True)
    else:
        Ax[i].autoscale(enable=True, axis='both', tight=True)
    Ax[i].locator_params(axis='y',nbins=4)


In [13]:
#	Prepare sum of energies

E_summ = []
for k in range(len(E_iValues)):
    E_summ.append(np.zeros(last, dtype='f8'))
for i in range(nPlasmas):
    for k in range(len(E_iValues)):
        E_summ[k] += balances[i][-last:,E_iValues[k]]

E_summ[3] -= balances[0][-last:,E_iValues[3]]							#remove electron rad which alread contains all rads

E_summ.append(E_summ[0] - E_summ[1] + E_summ[2]) 	#Ediff
if(use_time == 1):
    E_summ.append(np.append(0, 0.5*np.cumsum((E_summ[4][1:] + E_summ[4][:-1])*(x_balances[1:]-x_balances[:-1]))))	#summ of 	energy variation
    E_summ[-1] += E_summ[5][0]
    E_labels.append("$Es_{var}$")
    E_summ.append(np.append(0, 0.5*np.cumsum((E_summ[6][1:] + E_summ[6][:-1])*(x_balances[1:]-x_balances[:-1]))))	#summ of 	energy variation
    E_summ[-1] += E_summ[5][0]
    E_labels.append("$Es_{diff}$")

Ax[0].set_title(os.path.basename(os.path.abspath(path))+" @ t={:.3f} s".format(tempus))

for i in range(len(R_iValues)):
    Ax[0].plot(x_balances, residuals[-last:,R_iValues[i]]*R_facts[i], R_lines[i], label=R_labels[i])

all_ions = "e"
for Atom in Atoms: all_ions = all_ions + " - " + Atom
Ax[1].set_title(all_ions)
for i in range(len(E_summ)):
    Ax[1].plot(x_balances, E_summ[i]*E_facts[i], E_lines[i], label=E_labels[i])

Ax[2].set_title(ions[1])
for i in range(len(N_iValues)):
    Ax[2].plot(x_balances, balances[1][-last:,N_iValues[i]]*N_facts[i], N_lines[i], label=N_labels[i])

Ax[3].set_title(ions[0])
Ax[4].set_title(ions[1])
for i in range(len(E_iValues)):
    Ax[3].plot(x_balances,  balances[0][-last:,E_iValues[i]]*E_facts[i], E_lines[i], label=E_labels[i])
    Ax[4].plot(x_balances,  balances[1][-last:,E_iValues[i]]*E_facts[i], E_lines[i], label=E_labels[i])

In [53]:
df = pd.DataFrame()
for i in range(len(R_iValues)):
    df[R_labels[i]] = residuals[-last:,R_iValues[i]]*R_facts[i]
    
for i in range(len(N_iValues)):
    df[N_labels[i]] = balances[1][-last:,N_iValues[i]] # Do not multiply by N_facts to maintain SI
    
for i in range(len(E_summ)):
    df[E_labels[i]+"_tot"] = E_summ[i] * E_facts[i]   # Keep factors to stay in [MW]
    
for i in range(len(E_iValues)):
    df[E_labels[i]+"_e"] = balances[0][-last:,E_iValues[i]]*E_facts[i] # Keep factors to stay in [MW]
    df[E_labels[i]+"_i"] = balances[1][-last:,E_iValues[i]]*E_facts[i] # Keep factors to stay in [MW]


In [54]:
df

,$Res_Ne*10^{-3}$,$Res_ND*10^{-3}$,$Res_Ge*10^{-3}$,$Res_GD*10^{-3}$,$Res_{Te}*10^{-3}$,$Res_{TD}*10^{-3}$,$\Gamma_{in}*10^{21}$,$\Gamma_{out}*10^{24}$,$\Gamma_{sou}*10^{24}$,$N_{Var}*10^{22}$,...,$P_{out}$_e,$P_{out}$_i,$P_{sou}$_e,$P_{sou}$_i,$P_{rad}$_e,$P_{rad}$_i,$P_{var}$_e,$P_{var}$_i,$E_{stored}$_e,$E_{stored}$_i
0,0.005707,0.005707,0.012949,0.012949,0.000716,0.024069,6.988139e+20,1.949833e+22,1.880250e+22,4.485141e+18,...,0.617319,0.690630,-0.139821,-0.300606,0.0,0.0,0.002893,0.008778,0.010126,0.010455
1,0.004810,0.004810,0.019742,0.019742,0.000338,0.004844,6.988139e+20,1.949809e+22,1.880224e+22,4.472139e+18,...,0.617209,0.690600,-0.139819,-0.300591,0.0,0.0,0.003005,0.008847,0.010126,0.010455
2,0.004081,0.004081,0.023161,0.023161,0.000211,0.029273,6.988139e+20,1.949791e+22,1.880204e+22,4.449152e+18,...,0.617154,0.690584,-0.139816,-0.300580,0.0,0.0,0.003063,0.008843,0.010126,0.010455
3,0.003482,0.003482,0.024403,0.024403,0.000181,0.026606,6.988139e+20,1.949777e+22,1.880189e+22,4.438394e+18,...,0.617120,0.690574,-0.139814,-0.300573,0.0,0.0,0.003099,0.008897,0.010126,0.010455
4,0.002992,0.002992,0.024263,0.024263,0.000155,0.026906,6.988139e+20,1.949765e+22,1.880176e+22,4.429680e+18,...,0.617096,0.690567,-0.139812,-0.300567,0.0,0.0,0.003125,0.008898,0.010126,0.010455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.001063,0.001063,0.004685,0.004685,0.000162,0.018912,7.004583e+20,1.950204e+22,1.880166e+22,1.584334e+18,...,0.616900,0.690399,-0.139824,-0.302266,0.0,0.0,0.003308,0.007403,0.010127,0.010455
996,0.001033,0.001033,0.004661,0.004661,0.000161,0.028719,7.004583e+20,1.950207e+22,1.880170e+22,1.584324e+18,...,0.616899,0.690398,-0.139824,-0.302268,0.0,0.0,0.003311,0.007365,0.010127,0.010455
997,0.001004,0.001004,0.004632,0.004632,0.000115,0.006193,7.004583e+20,1.950211e+22,1.880173e+22,1.585280e+18,...,0.616897,0.690397,-0.139823,-0.302269,0.0,0.0,0.003313,0.007380,0.010127,0.010455
998,0.000976,0.000976,0.004597,0.004597,0.000113,0.043161,7.004583e+20,1.950214e+22,1.880176e+22,1.584512e+18,...,0.616895,0.690396,-0.139823,-0.302271,0.0,0.0,0.003315,0.007346,0.010127,0.010455


In [49]:

    
e_map

{'$P_{in}$_tot': 'P_in_tot',
 '$P_{in}$_i': 'P_in_i',
 '$P_{in}$_e': 'P_in_e',
 '$P_{out}$_tot': 'P_out_tot',
 '$P_{out}$_i': 'P_out_i',
 '$P_{out}$_e': 'P_out_e',
 '$P_{sou}$_tot': 'P_sou_tot',
 '$P_{sou}$_i': 'P_sou_i',
 '$P_{sou}$_e': 'P_sou_e',
 '$P_{rad}$_tot': 'P_rad_tot',
 '$P_{rad}$_i': 'P_rad_i',
 '$P_{rad}$_e': 'P_rad_e',
 '$P_{var}$_tot': 'P_var_tot',
 '$P_{var}$_i': 'P_var_i',
 '$P_{var}$_e': 'P_var_e',
 '$E_{stored}$_tot': 'E_stored_tot',
 '$E_{stored}$_i': 'E_stored_i',
 '$E_{stored}$_e': 'E_stored_e',
 '$P_{DIFF}$_tot': 'P_DIFF_tot',
 '$P_{DIFF}$_i': 'P_DIFF_i',
 '$P_{DIFF}$_e': 'P_DIFF_e'}

In [45]:
df

,$\Gamma_{in}*10^{21}$,$\Gamma_{out}*10^{24}$,$\Gamma_{sou}*10^{24}$,$N_{Var}*10^{22}$,$N_{Stored}*10^{21}$,$P_{in}$_tot,$P_{out}$_tot,$P_{sou}$_tot,$P_{rad}$_tot,$P_{var}$_tot,$E_{stored}$_tot,$P_{DIFF}$_tot
0,6.988139e+20,1.949833e+22,1.880250e+22,4.485141e+18,4.058952e+19,1.760076,1.307949,-0.440427,0.0,0.011670,0.020581,0.011700
1,6.988139e+20,1.949809e+22,1.880224e+22,4.472139e+18,4.058952e+19,1.760076,1.307809,-0.440409,0.0,0.011852,0.020581,0.011857
2,6.988139e+20,1.949791e+22,1.880204e+22,4.449152e+18,4.058952e+19,1.760076,1.307738,-0.440396,0.0,0.011906,0.020581,0.011942
3,6.988139e+20,1.949777e+22,1.880189e+22,4.438394e+18,4.058952e+19,1.760076,1.307694,-0.440386,0.0,0.011997,0.020581,0.011996
4,6.988139e+20,1.949765e+22,1.880176e+22,4.429680e+18,4.058952e+19,1.760076,1.307663,-0.440379,0.0,0.012023,0.020581,0.012034
...,...,...,...,...,...,...,...,...,...,...,...,...
995,7.004583e+20,1.950204e+22,1.880166e+22,1.584334e+18,4.058942e+19,1.760076,1.307299,-0.442090,0.0,0.010711,0.020581,0.010686
996,7.004583e+20,1.950207e+22,1.880170e+22,1.584324e+18,4.058942e+19,1.760076,1.307296,-0.442091,0.0,0.010676,0.020581,0.010688
997,7.004583e+20,1.950211e+22,1.880173e+22,1.585280e+18,4.058942e+19,1.760076,1.307294,-0.442092,0.0,0.010693,0.020581,0.010690
998,7.004583e+20,1.950214e+22,1.880176e+22,1.584512e+18,4.058942e+19,1.760076,1.307291,-0.442093,0.0,0.010661,0.020581,0.010692
